In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statistics import mean

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px

In [ ]:
df=pd.read_csv('../input/men-women-shoes-reviews/Shoes_Data.csv')

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
datatypes = df.dtypes
datatypes

In [ ]:
def rating_numeric(row):
    ratnum = row['rating'].split(' ')[0]
    return ratnum

df['rating_numeric'] = df.apply(lambda x: rating_numeric(x), axis=1)
df['rating_numeric'] = df.rating_numeric.astype(float)

In [ ]:
df.head(5)

In [ ]:
df.dtypes

In [ ]:
df["price"]=df["price"].apply(lambda x: str(x).replace('₹','') if '₹' in str(x) else str(x))
df["price"] = df.price.astype(float)

#add summary of price of shoes

In [ ]:
men_count = (df['Shoe Type']=='Men').sum()
#print(men_count)
women_count = (df['Shoe Type']=='Women').sum()
#print(women_count)

toGraph = {'shoe type': ['Men', 'Women'], 'count': [men_count, women_count]}
dftoGraph = pd.DataFrame(data=toGraph)
fig = px.bar(dftoGraph, x='shoe type', y='count', title='Shoe Count by Gender')
fig.show()



1. Scatter plot of ratings for mens and womens shoes (from rating and Shoe type column)

In [ ]:
fig = px.scatter(df, y="Shoe Type", x="rating_numeric", title="Range of ratings for mens + womens shoes")
fig.show()

2. Scatterplot of rating and price with color being Shoe Type

In [ ]:
fig = px.scatter(df, y="price", x="rating_numeric", color="Shoe Type", title="Rating vs Price (rupees) for Mens + Womens shoes")
fig.show()

In [ ]:
gender = df.groupby('Shoe Type').agg('mean')
gender = gender.round(1)
gender = gender.reset_index()

fig = px.bar(gender, x='Shoe Type', y='price', title=' AVG Price of Shoe by Gender')
fig.show()
fig = px.bar(gender, x='Shoe Type', y='rating_numeric', title='AVG Rating of Shoe by Gender')
fig.show()


In [ ]:
import re
tot = (' ').join(df['title'])
totarr = tot.split(' ')
def tofilter(word):
    invalidwords = ['shoes', 'shoe', 'men\'s','mens', 'women', 'womens', 'women\'s', 'men', 'women' 'sneakers', 'sneaker']
    if word.islower():
        return False
    if word.lower() in invalidwords or  word=='Sneakers':
        return False
    if re.search('[a-zA-Z]', word):
        return True
    return False
        
    
filtering = filter(tofilter, totarr)
features = list(filtering)


In [ ]:
from collections import Counter
counters = Counter(features)
top10 = counters.most_common(10)

In [ ]:
featuredf = pd.DataFrame(top10, columns =['Feature', 'Frequency'])
featuredf.head(5)

In [ ]:
fig = px.bar(featuredf, x='Feature', y='Frequency', title='Frequency of Shoe Mentions')
fig.show()

In [ ]:
def rating_feature(feature_row):
    ind = pd.Index(df['title'])
    arr = ind.str.contains(feature_row['Feature'], regex=False)
    l = []
    for i in range(len(arr)):
        if arr[i]:
            l.append(df.iloc[i]['rating_numeric'])
    return mean(l)
    
            




In [ ]:
featuredf['rating'] = featuredf.apply(lambda x: rating_feature(x), axis=1)

In [ ]:
featuredf['rating'] = featuredf['rating'].round(1)

In [ ]:
fig = px.bar(featuredf, x='Feature', y='rating', title='Ratings Associated with Shoe Features')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

NOTE: This graph is not an endorsement of any one type of shoe nor is it indicative of their performance. It is only the ratings associated with the feature MENTIONS in the title column of the dataset. Feature mentions do not necessarily mean the feature is part of the shoe.

Improvements for future
1. word cloud for reviews by feature
2. use natural language processing
3. learn details of natural language processing 

In [ ]:
# toadd = []
# for f, feature in enumerate(features):
#     ind = pd.Index(df['title'])
#     arr = ind.str.contains(feature, regex=False)
#     s=""
#     count = 0
#     for i in range(len(arr)):
#         if arr[i]:
#             s=s+df.iloc[i]['reviews']
#             count=count+1
#         if count==5:
#             break
#     toadd.append(s)
# featuredf['some reviews']=toadd
            
            
    